# **5400 Term Project** 

## Ingredients - Recipe Converter


Group 4: Jiarong Zhang, Weiyi Xia, Shuning Xu, Shiyu Hua, Zeying Liu

### Background

Our group aimed to produce a recipe search engine so that users can find the desire recipe and its relavent information based solely on ingredients used.

Our goal:

- Time Control 
- Plan, cook and share meals
- Discover healthy and economical eating habits
- Reduce food wastage

Our target user:

- People who have lots of ingredients in the refrigerator but can’t decide what to do today
- People who don’t know the formal name of specific dishes but only have some ingredients
- People who need to eliminate all ingredients left in the refrigerator ASAP
- People who have fitness or fat loss goals and pay attention to balanced nutrition


### Load the data source

In [2]:
import pandas as pd
import numpy as np
import json
from neo4j import GraphDatabase
from elasticsearch_dsl import connections
from elasticsearch.helpers import bulk
from elasticsearch_dsl import Search
from pprint import pprint
import string
from flask import Flask, request, render_template

In [3]:
with open('C:/Users/lilyh/Desktop/second semester/5400/group project/data.json') as f:
    records = json.load(f)
print('There are', len(records), 'records in the data source.')

There are 49698 records in the data source.


In [4]:
# display the first record
pprint(records[0])

{'Calcium_mg': 11.0,
 'Calories from Fat_kcal': 212.0,
 'Calories_kcal': 308.0,
 'Carbohydrates_g': 1.8,
 'Cholesterol_mg': 62.0,
 'Cook': '2h45m',
 'Dietary Fiber_g': 0.5,
 'Directions': 'Preheat oven to 200 degrees F (95 degrees C).;Season pork '
               'belly with paprika, salt, and pepper. Tightly wrap pork twice '
               'in heavy-duty aluminum foil. Place on a baking sheet and bake '
               'in the preheated oven for 2 1/2 hours. Turn off the oven; let '
               'pork rest in the oven for 1 hour. Remove meat from oven, '
               'leaving it wrapped in aluminum foil, and refrigerate at least '
               '8 hours or overnight.;Remove pork from foil and slice across '
               'the grain in 1/4-inch thick slices. Working in batches, cook '
               'pork in a non-stick skillet over medium heat until golden and '
               'crisped, 6 to 8 minutes per slice.',
 'Fat_g': 23.6,
 'Folate_mcg': 2.0,
 'Iron_mg': 1.0,
 'Magnesium_

### Neo4j Graphic Database

##### connect to the database

In [5]:
database_name = "neo4j"
username = "neo4j"
password = "apan5400"
uri = "bolt://localhost:7687/" + database_name

driver = GraphDatabase.driver(uri, auth=(username, password))
session = driver.session()

print("Successfully connected to Neo4j!")

Successfully connected to Neo4j!


In [6]:
query = (
    "LOAD CSV WITH HEADERS FROM 'file:///Users/lilyh/Desktop/ingre_recipe.csv' AS row "
     "MERGE (r:Recipe {recipeId: row.recipe_id})"
     "MERGE (i:Ingredient {ingredientName: row.ingredient})"
     "MERGE (r)-[:LINKED_TO]->(i)"
    )
result = session.run(query)
print("All ingredients and recipes are imported from the dataframe!")

All ingredients and recipes are imported from the dataframe!


##### Use neo4j to query individual ingredient and return a list of recipe_id connection 

In [7]:
class MyTemplate(string.Template):
    delimiter = '%'
    idpattern = '[a-z]+'

def queryData(userInput):
    templateText =  '''
      MATCH (ingredient:Ingredient {ingredientName: '%input'})-[:LINKED_TO]-(recipe:Recipe) RETURN recipe.recipeId as recipeId
    '''
        
    t = MyTemplate(templateText)
    d = {'input': userInput}
    stmt = (t.safe_substitute(d))
    res = session.run(stmt)
    
    recipe_id_2 = []
    for record in res:
        recipe_id_2.append(int(record["recipeId"]))
    
    return recipe_id_2

##### show the first 5 recipe_id use chicken wings as one of the ingredients

In [96]:
queryData('chicken wings')[0:5]

[25187, 229774, 256752, 60617, 233334]

### Elasticsearch querying

##### connect to elasticsearch and bulk index records

In [9]:
client = connections.create_connection(hosts=["localhost"],
                     port=9200, 
                     http_auth=('elastic', 'elastic'), 
                     ca_certs='C:/Users/lilyh/http_ca.crt',
                     use_ssl=True, 
                     verify_certs=True)

In [10]:
resp = bulk(client, records, index = "recipe")

##### simple query show recipe names based on rate (4-4.5)

In [11]:
s= Search(using = client, index = "recipe")  \
        .filter('range', aver_rate = {'gte': 4.0, 'lte': 4.5}) 

response = s.execute() 

for hit in response:
    print(hit.recipe_name)

Potato Bacon Pizza
Reuben Sandwich I
Cranberry Pork Chops II
Zucchini Noodles and Summer Vegetables with Sweet Pepper Chicken Sausage
Fresh Balsamic Chicken Salad
Hearty Chili
Tater Tot Casserole III
Slow Cooker Veggie-Beef Soup with Okra
Red Wine Braised Short Ribs with Rosemary
Nanny's Goulash


### Flask interactive output

##### Test trial

In [93]:
app = Flask("project")

def rate_value(x):
        if x == '1-2':
            return [1.00,2.00]
        elif x == '2-3':  
            return [2.00,3.00]
        elif x == '3-4':
            return [3.00,4.00]
        elif x == '4-5':
            return [4.00,5.00]
        else:
            return [0.00,1.00]

def calory_value(x):
        if x == '150-300':
            return [150.00,300.00]
        elif x == '300-450':
            return [300.00,450.00]
        elif x == '<150':
            return [0.00,150.00]
        elif x == '>450':
            return [450.00,1000.00]

def fat_value(x):
        if x == '5-15':
            return [5.00,15.00]
        elif x == '15-25':
            return [15.00,25.00]
        elif x == '<5':
            return [0.00,5.00]
        elif x == '>25':
            return [25.00,500.00]

def sugar_value(x):
        if x == '3-6':
            return [3.00,6.00]
        elif x == '6-15':
            return [6.00,15.00]
        elif x == '<3':
            return [0.00,3.00]
        elif x == '>15':
            return [15.00,100.00]

def sodium_value(x):
        if x == '100-300':
            return [100.00,300.00]
        elif x == '300-700':
            return [300.00,700.00]
        elif x == '<100':
            return [0.00,100.00]
        elif x == '>700':
            return [700.00,2000.00]

def calcium_value(x):
        if x == '25-60':
            return [25.00,60.00]
        elif x == '60-120':
            return [60.00,120.00]
        elif x == '<25':
            return [0.00,25.00]
        elif x == '>120':
            return [120.00,500.00]

def protein_value(x):
        if x == '3-10':
            return [3.00,10.00]
        elif x == '10-20':
            return [10.00,20.00]
        elif x == '<3':
            return [0.00,3.00]
        elif x == '>20':
            return [20.00,500.00]
        

@app.route('/', methods=['GET', 'POST'])
def recipe_search():
    if request.method == "POST":
        #variables
        ingredient = str(request.form.get('ingredient'))
        rate = str(request.form.get('aver_rate'))
        calory = str(request.form.get('Calories-kcal'))
        fat = str(request.form.get('Fat-g'))
        sugar = str(request.form.get('Sugars-g'))
        sodium = str(request.form.get('Sodium-mg'))
        calcium = str(request.form.get('Calcium-mg'))
        protein = str(request.form.get('Protein-g'))
        
        res = queryData(ingredient)
        s = Search(using = client, index = "recipe")  \
        .filter('terms', recipe_id = res) \
        .filter('range', aver_rate = {'gte': rate_value(rate)[0], 'lte': rate_value(rate)[1]}) \
        .filter('range', Calories_kcal = {'gte': calory_value(calory)[0], 'lte': calory_value(calory)[1]}) \
        .filter('range', Calcium_mg = {'gte':calcium_value(calcium)[0], 'lte':calcium_value(calcium)[1]}) \
        .filter('range', Fat_g = {'gte': fat_value(fat)[0], 'lte': fat_value(fat)[1]}) \
        .filter('range', Protein_g = {'gte':protein_value(protein)[0], 'lte': protein_value(protein)[1]}) \
        .filter('range', Sodium_mg = {'gte': sodium_value(sodium)[0], 'lte': sodium_value(sodium)[1]}) \
        .filter('range', Sugars_g = {'gte': sugar_value(sugar)[0], 'lte': rate_value(sugar)[1]}) \
        .sort({'aver_rate': {'order': 'desc'}})
        
        response = s.execute()
        
        if len(response['hits']['hits']) != 0:
            for hit in response['hits']['hits']:
                name = hit['_source']['recipe_name']
                direction = hit['_source']['Directions']
                cook = hit['_source']['Cook']
                image = hit['_source']['image_url']
                calories_kcal = hit['_source']['Calories_kcal']
                calcium_mg = hit['_source']['Calcium_mg']
                fat_g = hit['_source']['Fat_g']
                protein_g = hit['_source']['Protein_g']
                sodium_mg = hit['_source']['Sodium_mg']
                sugars_g = hit['_source']['Sugars_g']
                aver_rate = hit['_source']['aver_rate']
            return "Recipe name: "+name+". Directions: "+direction+". Cook time: "+cook+ \
                    ". Calgories_kcal: "+str(calories_kcal)+". Calcium_mg: "+str(calcium_mg)+". Fat_g:"+str(fat_g)+ \
                    ". Protein_g: "+str(protein_g)+". Sodium_mg: "+str(sodium_mg)+". Sugars_g: "+str(sugars_g)+ \
                    ". Rate: "+str(aver_rate)+". Image url: "+image
        else :
            return "No result. "
    
    return render_template('Index.html')

## flask can only return one hit 

In [94]:
app.run(host='localhost', port=5003)

 * Serving Flask app "project" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:5003/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Apr/2022 16:43:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Apr/2022 16:43:35] "POST / HTTP/1.1" 200 -


##### with output format

In [98]:
app = Flask("project")

def rate_value(x):
        if x == '1-2':
            return [1.00,2.00]
        elif x == '2-3':  
            return [2.00,3.00]
        elif x == '3-4':
            return [3.00,4.00]
        elif x == '4-5':
            return [4.00,5.00]
        else:
            return [0.00,1.00]

def calory_value(x):
        if x == '150-300':
            return [150.00,300.00]
        elif x == '300-450':
            return [300.00,450.00]
        elif x == '<150':
            return [0.00,150.00]
        elif x == '>450':
            return [450.00,1000.00]

def fat_value(x):
        if x == '5-15':
            return [5.00,15.00]
        elif x == '15-25':
            return [15.00,25.00]
        elif x == '<5':
            return [0.00,5.00]
        elif x == '>25':
            return [25.00,500.00]

def sugar_value(x):
        if x == '3-6':
            return [3.00,6.00]
        elif x == '6-15':
            return [6.00,15.00]
        elif x == '<3':
            return [0.00,3.00]
        elif x == '>15':
            return [15.00,100.00]

def sodium_value(x):
        if x == '100-300':
            return [100.00,300.00]
        elif x == '300-700':
            return [300.00,700.00]
        elif x == '<100':
            return [0.00,100.00]
        elif x == '>700':
            return [700.00,2000.00]

def calcium_value(x):
        if x == '25-60':
            return [25.00,60.00]
        elif x == '60-120':
            return [60.00,120.00]
        elif x == '<25':
            return [0.00,25.00]
        elif x == '>120':
            return [120.00,500.00]

def protein_value(x):
        if x == '3-10':
            return [3.00,10.00]
        elif x == '10-20':
            return [10.00,20.00]
        elif x == '<3':
            return [0.00,3.00]
        elif x == '>20':
            return [20.00,500.00]

@app.route('/', methods=['GET', 'POST'])
def home():
    if request.method == "POST":
        ingredient = str(request.form.get('ingredient'))
        rate = str(request.form.get('aver_rate'))
        calory = str(request.form.get('Calories-kcal'))
        fat = str(request.form.get('Fat-g'))
        sugar = str(request.form.get('Sugars-g'))
        sodium = str(request.form.get('Sodium-mg'))
        calcium = str(request.form.get('Calcium-mg'))
        protein = str(request.form.get('Protein-g'))
        
        res = queryData(ingredient)
        s = Search(using = client, index = "recipe")  \
        .filter('terms', recipe_id = res) \
        .filter('range', aver_rate = {'gte': rate_value(rate)[0], 'lte': rate_value(rate)[1]}) \
        .filter('range', Calories_kcal = {'gte': calory_value(calory)[0], 'lte': calory_value(calory)[1]}) \
        .filter('range', Calcium_mg = {'gte':calcium_value(calcium)[0], 'lte':calcium_value(calcium)[1]}) \
        .filter('range', Fat_g = {'gte': fat_value(fat)[0], 'lte': fat_value(fat)[1]}) \
        .filter('range', Protein_g = {'gte':protein_value(protein)[0], 'lte': protein_value(protein)[1]}) \
        .filter('range', Sodium_mg = {'gte': sodium_value(sodium)[0], 'lte': sodium_value(sodium)[1]}) \
        .filter('range', Sugars_g = {'gte': sugar_value(sugar)[0], 'lte': rate_value(sugar)[1]}) \
        .sort({'aver_rate': {'order': 'desc'}})
        
        response = s.execute()
        

        recipe_name = [hit.recipe_name for hit in response]
        directions = [hit.Directions for hit in response]
        cook = [hit.Cook for hit in response]
        rate = [hit.aver_rate for hit in response]
        calories_kcal = [hit.Calories_kcal for hit in response]
        calcium_mg = [hit.Calcium_mg for hit in response]
        fat_g = [hit.Fat_g for hit in response]
        protein_g = [hit.Protein_g for hit in response]
        sodium_mg = [hit.Sodium_mg for hit in response]
        sugars_g = [hit.Sugars_g for hit in response]
        image = [hit.image_url for hit in response]
        
        recipes = pd.DataFrame(list(zip(recipe_name, directions, cook, rate, calories_kcal, calcium_mg, fat_g, protein_g, sodium_mg, sugars_g, image)), 
                              columns = ['recipe_name', 'Directions','Cook','aver_rate','calories_kcal','calcium_mg','fat_g','protein_g','sodium_mg','sugars_g','image_url'])

    
    return render_template('home.html', recipe_name = recipe_name, directions = directions, cook = cook, rate = rate, calories_kcal = calories_kcal, 
                          calcium_mg = calcium_mg, fat_g = fat_g, protein_g = protein_g, sodium_mg = sodium_mg, sugars_g = sugars_g, image = image)




In [97]:
app.run(host='localhost', port=5004)

 * Serving Flask app "project" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:5004/ (Press CTRL+C to quit)
